In [ ]:
import numpy as np
from batchNormalization import BatchNormalization
from affine import Affine
from relu import Relu
from dropout import Dropout
from identityWithLoss import IdentityWithLoss
from model import Model
from optimizer import SGD, Momentum, AdaGrad, RMSprop, Adam
from trainer import Trainer
from util import scale
from metrics import Factory

In [ ]:
import pandas as pd
columns = {'CRIM':float, 'ZN':float, 'INDUS':float, 'CHAS':int, 'NOX':float, 'RM':float, 'AGE':float, 'DIS':float, 'RAD':object, 'TAX':int, 'PTRATIO':float, 'B':float, 'LSTAT':float, 'TARGET':float}
df = pd.read_csv('./housing.data', header=None, sep='\s+', na_values='na', names=columns.keys(), dtype=columns)
y = df['TARGET']
tp = df[['CHAS','RAD']]
df = df.drop(['TARGET','CHAS','RAD'], axis=1)
df = (df - df.mean()) / df.std(ddof=0)
tp = pd.get_dummies(tp, dummy_na=False, columns=['RAD'])
df = pd.concat([df,tp], axis=1, sort=False)
df.head()

In [ ]:
from sklearn.model_selection import ShuffleSplit

ss = ShuffleSplit(n_splits=1, train_size=0.7, random_state=0)
train_idx, test_idx = next(ss.split(df, y))
df = df.to_numpy()
y = y.to_numpy()
x_train, x_test, t_train, t_test = df[train_idx], df[test_idx], y[train_idx], y[test_idx]

In [ ]:
#学習
m = Model()
m.append(Affine(scale(21) * np.random.randn(21, 100), np.zeros(100)))
m.append(BatchNormalization(np.ones(100), np.zeros(100)))
m.append(Relu())
m.append(Dropout())
m.append(Affine(scale(100) * np.random.randn(100, 100), np.zeros(100)))
m.append(BatchNormalization(np.ones(100), np.zeros(100)))
m.append(Relu())
m.append(Dropout())
m.append(Affine(scale(100) * np.random.randn(100, 1), np.zeros(1)))
m.append_loss(IdentityWithLoss())
m.summary()

In [ ]:
t = Trainer(m, SGD(lr=0.1), Factory.create(Factory.MTYPE.R2))
t.fit(x_train, t_train, 30, 50)

In [ ]:
#テスト
for i in range(len(x_test)):
    y = m.predict(x_test[[i]])
    loss, prd = m.loss.forward(y, t_test[[i]])
    t.metrics.add_metrics(loss, t_test[i], prd)
t.metrics.get_metrics()